In [1]:
import os
from franz.openrdf.connect import ag_connect
from franz.openrdf.rio.rdfformat import RDFFormat
import pandas as pd
import geopandas as gpd
import ipyleaflet

pd.options.display.max_colwidth = 0

# Using GeoSPARQL in AllegroGraph

GeoSPARQL is a standard for representation and querying of geospatial linked data for the Semantic Web from the Open Geospatial Consortium (OGC). It provides:

* a vocabulary for representing geospatial data in RDF 
* an extension to the SPARQL query language for processing geospatial data

In this tutorial, we will demostrate how to use GeoSPARQL in AllegroGraph and our Python APIs.

## 1. Introduction

GeoSPARQL provides many classes and properties to model and annotate geospatial data. The following is a simplified illustration of GeoSPARQL's ontology, including [Feature](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#_class_geofeature), [Geometry](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#_class_geogeometry) and their properties. 

![](img/geosparql-ontology.png)

### 1.1 Feature and Geometry Class

Features and geometries are connected by using [geo:hasGeometry](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#_property_geohasgeometry), [geo:hasDefaultGeometry](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#_property_geohasdefaultgeometry) and etc. For example,

```turtle
@prefix : <http://example.org/ApplicationSchema#> .
@prefix geo: <http://www.opengis.net/ont/geosparql#> .

:Franz a geo:Feature.
:FranzGeom a geo:Geometry .
:Franz geo:hasGeometry :FranzGeom .
```

### 1.2 Serialization

Serializations annotate geomeries by using [geo:asWKT](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#_property_geoaswkt), [geo:asGeoJSON](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#_property_geoasgeojson) and etc. For example,

```turtle
@prefix : <http://example.org/ApplicationSchema#> .
@prefix geo: <http://www.opengis.net/ont/geosparql#> .

:FranzGeom geo:asWKT "POINT(-122.12956214302625 37.88963389303175)"^^geo:wktLiteral .
```

or in the equivalent GeoJSON format:

```turtle
@prefix : <http://example.org/ApplicationSchema#> .
@prefix geo: <http://www.opengis.net/ont/geosparql#> .

:FranzGeom geo:asGeoJSON '''{
  "type": "Point",
  "coordinates": [
    -122.12956214302625,
    37.88963389303175
  ]
}'''^^geo:geoJSONLiteral .
```

In the example above, both serializations represent [latitude: 37.88963389303175, longitude: -122.12956214302625](https://goo.gl/maps/ejrUhFSrJc8zRuadA). By standard, as shown below, a spatial reference identifier may be added at the beginning of a WKT literal, and `<http://www.opengis.net/def/crs/OGC/1.3/CRS84>` is used by default.

```turtle
:someGeom geo:asWKT """<http://www.opengis.net/def/crs/OGC/1.3/CRS84>
                 Point(-83.4 34.3)"""^^geo:wktLiteral .

:someOtherGeom geo:asWKT """<http://www.opengis.net/def/crs/EPSG/0/6366> 
                 Point(538447.8454476658 3602285.563945497)"""^^geo:wktLiteral .
```

Note that literals **must** be typed with [geo:wktLiteral](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#_rdfs_datatype_geowktliteral) or [geo:geoJSONLiteral](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#_rdfs_datatype_geogeojsonliteral).

To know more details of the WKT and GeoJSON format, please see:

* Well-known text representation of geometry, https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry
* GeoJSON - RFC 7946, https://tools.ietf.org/html/rfc7946

## 2. Basics

GeoSPARQL standard has [provided](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#_c_2_1_example_data) example data: [geosparql-example.ttl](data/geosparql-example.ttl), where these features and geometries are visualized in the figure below:

![](img/geosparql-example.png)

In this section, we'll use this data and demostrate some basics of using GeoSPARQL in AllegroGraph.

Let's set up the server connection's parameters first.

In [2]:
agraph_host = os.environ.setdefault("AGRAPH_HOST", "localhost")
agraph_port = os.environ.setdefault("AGRAPH_PORT", "10035")
agraph_user = os.environ.setdefault("AGRAPH_USER", "test")
agraph_password = os.environ.setdefault("AGRAPH_PASSWORD", "xyzzy")

print({
    "host": agraph_host,
    "port": agraph_port,
    "user": agraph_user,
    "password": agraph_password
})

{'host': 'localhost', 'port': '37209', 'user': 'test', 'password': 'xyzzy'}


Now we can create a repository and load triples.

In [3]:
conn = ag_connect('geosparql-example', clear=True, port=37209)
conn.addFile("data/geosparql-example.ttl", None, format=RDFFormat.TURTLE)
print("%d triples has been loaded." % conn.size())

43 triples has been loaded.


In the example dataset, it defines two subproperties, `my:hasExactGeometry` and `my:hasPointGeometry`:

```turtle
@prefix my: <http://example.org/ApplicationSchema#> .
@prefix geo: <http://www.opengis.net/ont/geosparql#> .

my:hasExactGeometry a rdf:Property ;
    rdfs:subPropertyOf geo:hasDefaultGeometry, geo:hasGeometry .

my:hasPointGeometry a rdf:Property ;
    rdfs:subPropertyOf geo:hasGeometry .
```

The query below will show you the "exact" geometry and "point" geometry of feature **my:A**:

In [4]:
conn.prepareTupleQuery(query="""
PREFIX my: <http://example.org/ApplicationSchema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>

SELECT ?aExactGeom ?aPointGeom 
WHERE {
  my:A my:hasExactGeometry ?aExactGeom ;
       my:hasPointGeometry ?aPointGeom .
}
""").evaluate().toPandas()

,aExactGeom,aPointGeom
0,<http://example.org/ApplicationSchema#AExactGeom>,<http://example.org/ApplicationSchema#APointGeom>


You can also look up the WKT serialization respectively.

In [5]:

conn.prepareTupleQuery(query="""
PREFIX my: <http://example.org/ApplicationSchema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>

SELECT ?aExactWKT ?aPointWKT 
WHERE {
  my:A my:hasExactGeometry [ geo:asWKT ?aExactWKT ] ;
       my:hasPointGeometry [ geo:asWKT ?aPointWKT ] .
}
""").evaluate().toPandas()



,aExactWKT,aPointWKT
0,"<http://www.opengis.net/def/crs/OGC/1.3/CRS84>\n Polygon((-83.6 34.1, -83.2 34.1, -83.2 34.5,\n -83.6 34.5, -83.6 34.1))",<http://www.opengis.net/def/crs/OGC/1.3/CRS84>\n Point(-83.4 34.3)


### 2.1 Simple Features Relation Family

As defined by Simple Features [[OGCSFACA]](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#OGCSFACA) [[ISO19125-1]](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#ISO19125-1), AllegroGraph supports `geof:sfEquals`, `geof:sfDisjoint`, `geof:sfIntersects`, `geof:sfTouches`, `geof:sfCrosses`, `geof:sfWithin`, `geof:sfContains` and `geof:sfOverlaps` as SPARQL extension functions, consistent with their corresponding DE-9IM intersection patterns, where `geof:` namespace has uri `http://www.opengis.net/def/function/geosparql/`.

All these functions take two arbitrary geometry serializations as arguments and return a boolean result. You may see [here](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#_simple_features_relation_family_2) for more details.

#### 2.1.1 geof:sfContains

By using `geof:sfContains`, we can find all features that feature **my:A** contains, 
where spatial calculations are based on **my:hasExactGeometry**.




In [6]:
conn.prepareTupleQuery(query="""
PREFIX my: <http://example.org/ApplicationSchema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>

SELECT ?f
WHERE {
    my:A my:hasExactGeometry ?aGeom .
    ?aGeom geo:asWKT ?aWKT .
    ?f my:hasExactGeometry ?fGeom .
    ?fGeom geo:asWKT ?fWKT .

    FILTER (
        geof:sfContains(?aWKT, ?fWKT) &&
            !sameTerm(?aGeom, ?fGeom)
        )
}
""").evaluate().toPandas()

,f
0,<http://example.org/ApplicationSchema#F>
1,<http://example.org/ApplicationSchema#B>


#### 2.1.2 geof:sfWithin

Find all features that are within a transient bounding box geometry, where spatial calculations are based on **my:hasPointGeometry**.



In [7]:
conn.prepareTupleQuery(query="""
PREFIX my: <http://example.org/ApplicationSchema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>

SELECT ?f
WHERE {
    ?f my:hasPointGeometry ?fGeom .
    ?fGeom geo:asWKT ?fWKT .
    FILTER (
        geof:sfWithin(
            ?fWKT,
            '''<http://www.opengis.net/def/crs/OGC/1.3/CRS84>
            Polygon ((-83.4 34.0, -83.1 34.0,
                        -83.1 34.2, -83.4 34.2,
                        -83.4 34.0))'''^^geo:wktLiteral
        )
    )
}
""").evaluate().toPandas()

,f
0,<http://example.org/ApplicationSchema#D>


### 2.2 Non-topological Query Functions

These functions include:

* geof:isEmpty
* geof:convexHull
* geof:intersection
* geof:union
* geof:difference
* geof:symDifference
* geof:envelope
* geof:centroid

#### 2.2.1 geof:union

[geof:union](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#_function_geofunion) returns a geometric object that represents all Points in the union of the given two geometric objects.

The example below finds all features that touch the union of feature **my:A** and feature **my:D**, where computations are based on **my:hasExactGeometry**.

In [8]:
conn.prepareTupleQuery(query="""
PREFIX my: <http://example.org/ApplicationSchema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>

SELECT ?f
WHERE {
    ?f my:hasExactGeometry ?fGeom .
    ?fGeom geo:asWKT ?fWKT .
    my:A my:hasExactGeometry ?aGeom .
    ?aGeom geo:asWKT ?aWKT .
    my:D my:hasExactGeometry ?dGeom .
    ?dGeom geo:asWKT ?dWKT .
    FILTER (
        geof:sfTouches(
            ?fWKT,
            geof:union(?aWKT, ?dWKT)
        )
    )
}
""").evaluate().toPandas()

,f
0,<http://example.org/ApplicationSchema#C>


#### 2.2.2 geof:distance

[geof:distance](https://opengeospatial.github.io/ogc-geosparql/geosparql11/spec.html#_function_geofdistance) returns the shortest distance in the given unit between any two Points in the two geometric objects. We use [QUDT ontology](https://qudt.org/) for representing units of measurements. Supported units are:

| Resource   | Unit of Measurement |
|------------|---------------------|
| qudt:M     | Meter               |
| qudt:KiloM | KiloMeter           |
| qudt:MI    | International Mile  |
| qudt:MI_US | Mile US Statute     |
| qudt:YD    | Yard                |
| qudt:FT    | Foot                |
| qudt:FT_US | US Survey Foot      |

The query below finds the 3 closest features to feature **my:C**, where computations are based on **my:hasExactGeometry**.

In [9]:
conn.prepareTupleQuery(query="""
PREFIX qudt: <http://qudt.org/vocab/unit/>
PREFIX my: <http://example.org/ApplicationSchema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>

SELECT ?f
WHERE {
    my:C my:hasExactGeometry ?cGeom .
    ?cGeom geo:asWKT ?cWKT .
    ?f my:hasExactGeometry ?fGeom .
    ?fGeom geo:asWKT ?fWKT .
    FILTER (?fGeom != ?cGeom)
}
ORDER BY ASC (geof:distance(?cWKT, ?fWKT, qudt:M))
LIMIT 3
""").evaluate().toPandas()

,f
0,<http://example.org/ApplicationSchema#A>
1,<http://example.org/ApplicationSchema#E>
2,<http://example.org/ApplicationSchema#D>


## 3. GeoSPARQL Extensions

On top of the standard GeoSPARQL functions, AllegroGraph adds a few useful extensions. In this section, the prefix `geoext:` stands for the namespace `http://franz.com/ns/allegrograph/3.0/geosparql/ext#`.

### 3.1 Geohash Indexing

[Geohash](https://en.wikipedia.org/wiki/Geohash) is an algorithm that encodes a geographic location into a short string of letters and digits. For example, [latitude: 37.88963389303175, longitude: -122.12956214302625](https://goo.gl/maps/ejrUhFSrJc8zRuadA) may be encoded as `"9q9psceun6"`. To provide better spatial query performance, we have implemented Geohash indexing, which is a critical component required by a few other extensions.

To index your spatial geometries, use either `geoext:buildGeohashIndex` or `geoext:rebuildGeohashIndex` [SPARQL magic property](https://franz.com/agraph/support/documentation/current/magic-properties.html).

#### 3.1.1 geoext:buildGeohashIndex

`geoext:buildGeohashIndex` **incrementally** builds Geohash index.
It only builds for triples that do NOT have geohash index.

For example:

```sparql
PREFIX geoext: <http://franz.com/ns/allegrograph/3.0/geosparql/ext#>
SELECT ?success WHERE {
    ?success geoext:buildGeohashIndex () .
}
```

If the building process finishes successfully, it will return `\"true\"^^<http://www.w3.org/2001/XMLSchema#boolean>`; otherwise, it will return (bound to `?success`) `\"false\"^^<http://www.w3.org/2001/XMLSchema#boolean>`.

#### 3.1.2 geoext:rebuildGeohashIndex

`geoext:buildGeohashIndex` **forcefully** rebuilds Geohash index.
It will delete all the current Geohash index and then rebuild.

For example:

```sparql
PREFIX geoext: <http://franz.com/ns/allegrograph/3.0/geosparql/ext#>
SELECT ?success WHERE {
    ?success geoext:rebuildGeohashIndex () .
}
```

If the rebuilding process finishes successfully, it will return `\"true\"^^<http://www.w3.org/2001/XMLSchema#boolean>`; otherwise, it will return (bound to `?success`) `\"false\"^^<http://www.w3.org/2001/XMLSchema#boolean>`.

#### 3.1.3 Example

We will use a different dataset `franz.ttl` to demostrate its usage. 
This dataset contains locations of Franz Inc. headquarter and a few nearby restaurants.

```turtle
@prefix : <http://example.org/ApplicationSchema#> .
@prefix geo: <http://www.opengis.net/ont/geosparql#> .

:Franz geo:hasGeometry :FranzGeom .
:FranzGeom geo:asWKT "POINT(-122.12956214302625 37.88963389303175)"^^geo:wktLiteral .

:OyamaSushi a :Restaurant ;
            geo:hasGeometry :OyamaSushiGeom .
:OyamaSushiGeom geo:asWKT "POINT(-122.1275488659957 37.89076272510303)"^^geo:wktLiteral .

:PostinoRestaurant a :Restaurant ;
            geo:hasGeometry :PostinoRestaurantGeom .
:PostinoRestaurantGeom geo:asWKT "POINT(-122.12107937769098 37.891406215514536)"^^geo:wktLiteral .

:TheCooperageAmericanGrille a :Restaurant ;
            geo:hasGeometry :TheCooperageAmericanGrilleGeom .
:TheCooperageAmericanGrilleGeom geo:asWKT "POINT(-122.12016742659232 37.89114373984338)"^^geo:wktLiteral .

:TheHideoutKitchenAndCafe a :Restaurant ;
            geo:hasGeometry :TheHideoutKitchenAndCafeGeom .
:TheHideoutKitchenAndCafeGeom geo:asWKT "POINT(-122.10956550861351 37.89419432704863)"^^geo:wktLiteral .

:TheParkBistroAndBar a :Restaurant ;
            geo:hasGeometry :TheParkBistroAndBarGeom .
:TheParkBistroAndBarGeom geo:asWKT "POINT(-122.1009540100645 37.89637380250877)"^^geo:wktLiteral .
```

Let's create an empty repository and load `franz.ttl` first.

In [10]:
conn = ag_connect('geosparql-example', clear=True, port=37209)
conn.addFile("data/franz.ttl", None, format=RDFFormat.TURTLE)
print("%d triples has been loaded." % conn.size())

17 triples has been loaded.


Let's build the Geohash index. 

In [11]:
conn.prepareTupleQuery(query="""
PREFIX geoext: <http://franz.com/ns/allegrograph/3.0/geosparql/ext#>

SELECT ?success WHERE {
    ?success geoext:buildGeohashIndex () .
}
""").evaluate().toPandas()

,success
0,True


After indexing, each geometry will be linked to its geohash by predicate `geoext:geohash`.

In [12]:
conn.prepareTupleQuery(query="""
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geoext: <http://franz.com/ns/allegrograph/3.0/geosparql/ext#>

SELECT ?place ?geometry ?geohash WHERE {
    ?place geo:hasGeometry ?geometry .
    ?geometry geoext:geohash ?geohash . # new triples added after indexing
}
""").evaluate().toPandas()

,place,geometry,geohash
0,<http://example.org/ApplicationSchema#OyamaSushi>,<http://example.org/ApplicationSchema#OyamaSushiGeom>,9q9pscv3ty
1,<http://example.org/ApplicationSchema#TheParkBistroAndBar>,<http://example.org/ApplicationSchema#TheParkBistroAndBarGeom>,9q9ptdbf3s
2,<http://example.org/ApplicationSchema#TheCooperageAmericanGrille>,<http://example.org/ApplicationSchema#TheCooperageAmericanGrilleGeom>,9q9pt1fgcv
3,<http://example.org/ApplicationSchema#Franz>,<http://example.org/ApplicationSchema#FranzGeom>,9q9pscegng
4,<http://example.org/ApplicationSchema#TheHideoutKitchenAndCafe>,<http://example.org/ApplicationSchema#TheHideoutKitchenAndCafeGeom>,9q9pt66t8v
5,<http://example.org/ApplicationSchema#PostinoRestaurant>,<http://example.org/ApplicationSchema#PostinoRestaurantGeom>,9q9pt1fjkw


### 3.2 Simple Features Relation Query by Magic Properties

In section 2.1, we have shown examples of doing relation query by using SPARQL extension functions. 
However, they are not using any Geohash indices hence their performance may be concerning if the dataset is fairly large.

In the meanwhile, GeoSPARQL provides their corresponding semantics in the form of RDF predicates, and we have implemented them as SPARQL magic properties. For example, `geof:sfWithin` corresponds to the predicate form `geo:sfWithin`. 

Note that these magic properties depends on Geohash indexing. Users may use these magic properties instead of the SPARQL extension functions when the performance is concerning.

#### 3.2.1 Example

Let's create an empty repository and load `geosparql-example.ttl`.

In [13]:
conn = ag_connect('geosparql-example', clear=True, port=37209)
conn.addFile("data/geosparql-example.ttl", None, format=RDFFormat.TURTLE)
print("%d triples has been loaded." % conn.size())

43 triples has been loaded.


Let's build the Geohash index.

In [14]:
conn.prepareTupleQuery(query="""
PREFIX geoext: <http://franz.com/ns/allegrograph/3.0/geosparql/ext#>

SELECT ?success WHERE {
    ?success geoext:buildGeohashIndex () .
}
""").evaluate().toPandas()

,success
0,True


Find all features that feature **my:A** contains, 
where spatial calculations are based on **my:hasExactGeometry**.

_It's just like section 2.1.1, but uses `geo:sfContains`._

**FIXME!**

In [15]:
conn.prepareTupleQuery(query="""
PREFIX my: <http://example.org/ApplicationSchema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>

SELECT ?f
WHERE {
    my:A my:hasExactGeometry ?aGeom .
    ?f my:hasExactGeometry ?fGeom .
    ?aGeom geo:sfContains ?fGeom .
    FILTER (!sameTerm(?aGeom, ?fGeom))
}
""").evaluate().toPandas()

,f


Find all features that are within a transient bounding box geometry, where spatial calculations are based on **my:hasPointGeometry**.

_It's just like section 2.1.2, but uses `geo:sfWithin`._

In [16]:
conn.prepareTupleQuery(query="""
PREFIX my: <http://example.org/ApplicationSchema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>

SELECT ?f
WHERE {
   ?f my:hasPointGeometry ?fGeom .
   ?fGeom geo:sfWithin '''
        <http://www.opengis.net/def/crs/OGC/1.3/CRS84>
            Polygon ((-83.4 34.0, -83.1 34.0,
                      -83.1 34.2, -83.4 34.2,
                      -83.4 34.0))
        '''^^geo:wktLiteral
}
""").evaluate().toPandas()

,f
0,<http://example.org/ApplicationSchema#D>


### 3.3 geoext:haversineDistance

`geoext:haversineDistance` is a SPARQL extension function to compute [haversine distance](https://en.wikipedia.org/wiki/Haversine_formula) from two geometric objects.

The example below shows the haversine distance (unit is Meter by default) between New York and London.

In [17]:
conn.prepareTupleQuery(query="""
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geoext: <http://franz.com/ns/allegrograph/3.0/geosparql/ext#>

SELECT ?haversineDistance
WHERE {
  BIND (geoext:haversineDistance("POINT(-74.006 40.7128)"^^geo:wktLiteral, # New York
                                 "POINT(-0.1278 51.5074)"^^geo:wktLiteral) # London
        AS ?haversineDistance)
}
""").evaluate().toPandas()

,haversineDistance
0,5.570230e+06


And just like `geof:distance`, you may specify the unit of measurements. For example, you can use `qudt:FT` to get the distance in Feet.

In [18]:
conn.prepareTupleQuery(query="""
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geoext: <http://franz.com/ns/allegrograph/3.0/geosparql/ext#>
PREFIX qudt: <http://qudt.org/vocab/unit/>

SELECT ?haversineDistanceFeet
WHERE {
  BIND (geoext:haversineDistance("POINT(-74.006 40.7128)"^^geo:wktLiteral, # New York
                                 "POINT(-0.1278 51.5074)"^^geo:wktLiteral, # London
                                qudt:FT)
        AS ?haversineDistanceFeet)
}
""").evaluate().toPandas()

,haversineDistanceFeet
0,1.827503e+07


### 3.4 geoext:nearby

`geoext:nearby` is a SPARQL magic property that returns all geometric objects that are within a specified radius of a given _Point_ serialization. **It depends on Geohash indexing.**

Let's load `franz.ttl` again and build Geohash index.

In [19]:
conn = ag_connect('franz', clear=True)
conn.addFile("data/franz.ttl", None, format=RDFFormat.TURTLE)

conn.prepareTupleQuery(query="""
PREFIX geoext: <http://franz.com/ns/allegrograph/3.0/geosparql/ext#>

SELECT ?success WHERE {
    ?success geoext:buildGeohashIndex () .
}
""").evaluate().toPandas()

,success
0,True


The query below lists all the restaurants' distances (km) to Franz office in ascending order.

In [20]:
conn.prepareTupleQuery(query="""
PREFIX : <http://example.org/ApplicationSchema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX qudt: <http://qudt.org/vocab/unit/>

SELECT ?restaurant ?distance
WHERE {
  :FranzGeom geo:asWKT ?franzLiteral .
  
  ?restaurant a :Restaurant;
         geo:hasGeometry ?restaurantGeom .
  ?restaurantGeom geo:asWKT ?restaurantLiteral . 
  
  BIND (geof:distance(?franzLiteral, ?restaurantLiteral, qudt:KiloM) AS ?distance)

}
ORDER BY ASC(?distance)
""").evaluate().toPandas()

,restaurant,distance
0,<http://example.org/ApplicationSchema#OyamaSushi>,0.216936
1,<http://example.org/ApplicationSchema#PostinoRestaurant>,0.771664
2,<http://example.org/ApplicationSchema#TheCooperageAmericanGrille>,0.843209
3,<http://example.org/ApplicationSchema#TheHideoutKitchenAndCafe>,1.830314
4,<http://example.org/ApplicationSchema#TheParkBistroAndBar>,2.625215


Search for restaurants that are within radius of 1 km. 

In [21]:
conn.prepareTupleQuery(query="""
PREFIX : <http://example.org/ApplicationSchema#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX geoext: <http://franz.com/ns/allegrograph/3.0/geosparql/ext#>
PREFIX qudt: <http://qudt.org/vocab/unit/>

SELECT ?restaurant ?distance
WHERE {
  :FranzGeom geo:asWKT ?franzWKT .
  
  ?restaurant a :Restaurant;
         geo:hasGeometry ?restaurantGeom .
  ?restaurantGeom geo:asWKT ?restaurantLiteral . 
  
  BIND (geof:distance(?franzWKT, ?restaurantLiteral, qudt:KiloM) AS ?distance)
  
  ?restaurantGeom geoext:nearby (?franzWKT 1 qudt:KiloM) .
}
ORDER BY ASC(?distance)
""").evaluate().toPandas()

,restaurant,distance
0,<http://example.org/ApplicationSchema#OyamaSushi>,0.216936
1,<http://example.org/ApplicationSchema#PostinoRestaurant>,0.771664
2,<http://example.org/ApplicationSchema#TheCooperageAmericanGrille>,0.843209


## 4. An Interactive Example

In this section, we will create an interactive map by using [Mobile Food Facility Permit](https://catalog.data.gov/dataset/mobile-food-facility-permit) dataset which includes name of vendor, location, type of food sold and status of permit. 
From the map, users can draw polygons and a callback will execute GeoSPARQL queries to find points that are within the created polygons.

Let's read `Mobile_Food_Facility_Permit.csv` by using Pandas and [GeoPandas](https://geopandas.org/en/stable/index.html) first.

In [22]:
# Read csv into a DataFrame
df = pd.read_csv(
    "data/Mobile_Food_Facility_Permit.csv",
    dtype={"locationid": str, "Longitude": float, "Latitude": float},    
)

# Filter "APPROVED" records
df = df[df.Status == "APPROVED"]

# Construct a GeoDataFrame from df
# "geometry" column is built from "Logitude" and "Latitude" columns
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

gdf.head()

,locationid,Applicant,FacilityType,cnn,LocationDescription,Address,blocklot,block,lot,permit,...,Received,PriorPermit,ExpirationDate,Location,Fire Prevention Districts,Police Districts,Supervisor Districts,Zip Codes,Neighborhoods (old),geometry
0,1571753,The Geez Freeze,Truck,887000,18TH ST: DOLORES ST to CHURCH ST (3700 - 3799),3750 18TH ST,3579006,3579,006,21MFF-00015,...,20210315,0,11/15/2022 12:00:00 AM,"(37.76201920035647, -122.42730642251331)",8.0,4.0,5.0,28862.0,3.0,POINT (-122.42731 37.76202)
1,1565571,MOMO INNOVATION LLC,Truck,3525000,CALIFORNIA ST: DAVIS ST to FRONT ST (100 - 199),101 CALIFORNIA ST,0263011,0263,011,21MFF-00089,...,20211022,0,11/15/2022 12:00:00 AM,"(37.792948952834664, -122.39809861316652)",4.0,1.0,10.0,28860.0,6.0,POINT (-122.39810 37.79295)
2,1565954,Treats by the Bay LLC,Truck,10624001,POST ST: MONTGOMERY ST to LICK PL (1 - 40),1 MONTGOMERY ST,0292002,0292,002,21MFF-00094,...,20211022,0,11/15/2022 12:00:00 AM,"(37.78924953407508, -122.40241859729358)",4.0,1.0,10.0,28854.0,6.0,POINT (-122.40242 37.78925)
3,1565594,MOMO INNOVATION LLC,Truck,9094000,MISSION ST: ANNIE ST to 03RD ST (663 - 699),667 MISSION ST,3722067,3722,067,21MFF-00090,...,20211022,0,11/15/2022 12:00:00 AM,"(37.7865580501799, -122.40103337534973)",12.0,2.0,9.0,28855.0,6.0,POINT (-122.40103 37.78656)
5,1565628,Treats by the Bay LLC,Truck,5170000,ELLIS ST: POWELL ST to CYRIL MAGNIN ST (100 - 148),120 ELLIS ST,0326005,0326,005,21MFF-00093,...,20211022,0,11/15/2022 12:00:00 AM,"(37.78561008636915, -122.40815476434545)",14.0,10.0,10.0,28852.0,36.0,POINT (-122.40815 37.78561)


Now we can start importing triples and building Geohash index.

In [23]:
conn = ag_connect('Mobile_Food_Facility_Permit', clear=True)

# Namespaces
ns = 'ex://food/'
ns_geo = "http://www.opengis.net/ont/geosparql#"
ns_geof = "http://www.opengis.net/def/function/geosparql/"
ns_geoext = "http://franz.com/ns/allegrograph/3.0/geosparql/ext#"
ns_qudt = "http://qudt.org/vocab/unit/"

# RDF Properties
name = conn.createURI(ns, 'name')
location = conn.createURI(ns, 'location')
geo_hasGeometry = conn.createURI(ns_geo, "hasGeometry")
geo_asWKT = conn.createURI(ns_geo, "asWKT")

# RDF Datatype
geo_wktLiteral = conn.createURI(ns_geo, "wktLiteral")

for _, row in gdf.iterrows():
    if row.Longitude != 0 and row.Latitude != 0:
        subject = conn.createURI(ns, row.locationid)
        geometry = conn.createURI(ns, row.locationid+"Geom")
        wkt_literal = conn.createLiteral(row.geometry.wkt, datatype=geo_wktLiteral)
        conn.addTriple(subject, name, conn.createLiteral(row.Applicant))
        conn.addTriple(subject, geo_hasGeometry, geometry)
        conn.addTriple(geometry, geo_asWKT, wkt_literal)
    
conn.prepareTupleQuery(query="""
PREFIX geoext: <http://franz.com/ns/allegrograph/3.0/geosparql/ext#>

SELECT ?success WHERE {
    ?success geoext:buildGeohashIndex () .
}
""").evaluate().toPandas()

,success
0,True


By using [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/index.html), we create an interactive map where all locations are marked.

In [24]:
# Initialize the map
m = ipyleaflet.Map(center=(37.7601167,-122.4580325), zoom=12)

# Add markers
for _, row in gdf.iterrows():
    m += ipyleaflet.Marker(location=(row.Latitude, row.Longitude), draggable=False)

# Draw polygon callback 
def handle_draw(self, action, geo_json):
    if action == 'created':
        polygon_geojson = json.dumps(geo_json["geometry"])
        query = f'''PREFIX : <ex://food/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>

SELECT DISTINCT ?s ?name WHERE
{{
    ?s :name ?name ;
        geo:hasGeometry ?geom .
    ?geom geo:sfWithin """{polygon_geojson}"""^^geo:geoJSONLiteral .
}}'''
        with conn.prepareTupleQuery(query=query).evaluate() as res:
            for bindings in res:
                print(bindings.getValue('s'), bindings.getValue("name"))

dc = ipyleaflet.DrawControl(polyline={}, circle={}, marker={}, edit=False)
dc.on_draw(handle_draw)
m.add_control(dc)
    
# Show the map    
m

Map(center=[37.7601167, -122.4580325], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

From the topleft column, we can click the "polygon" button and draw a polygon.

![](img/screenshot-1.jpg)

After drawing a polygon, `handle_draw` callback will print the contained points to log.

![](img/screenshot-2.jpg)
![](img/screenshot-3.jpg)